# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [17]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [22]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [23]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位资深的生物学专家。 你精通各个生物学分支领域，善于用通俗易懂的方式解答复杂的生命科学问题。 当遇到超出专业知识范畴的问题时，你会保持学术的谦逊并诚实回应。

这是一个问题： {input}"""

computer_science_template = """你是一位经验丰富的计算机科学导师。 你对算法、编程语言和软件开发等课题了如指掌，能够用简明的语言解决计算机科学相关的疑惑。 如果遇到不确定的问题，你会基于最新研究信息给出明智的推测。

这是一个问题： {input}"""

chinese_literature_template = """你是一位汉语言文学的专家和鉴赏家。 你对古今中外的文学作品都有深入的研究，并可以用深刻的见解来分析文学作品的各种元素。 当遇到无法立即回答的提问时，你会展现你的学者风范，委婉地表示需要更多时间来研究。

这是一个问题： {input}"""

In [24]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_science_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_literature_template,
    }
]

In [25]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", base_url="https://api.xiaoai.plus/v1")

In [26]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [27]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [28]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [29]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [30]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [31]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [32]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [33]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [34]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [35]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指理想的热辐射体所发出的辐射。它的特点是不同温度下发出的辐射具有不同的波长和强度分布。根据普朗克定律，黑体辐射的强度与波长成反比，即短波长的辐射强度更高。这个概念在研究物体的热辐射和能量转换中起着重要作用。'}


In [47]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '找出大于40的第一个质数，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '找出大于40的第一个质数，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要了解什么是质数。质数是指除了1和它本身以外，没有其他因数的数。根据这个定义，我们可以列出大于40的质数：41、43、47、53、59、61、67、71、73、79、83、89、97。\n\n接下来，我们需要找出这些质数中哪些能够满足条件：“加一能被3整除”。我们可以依次将这些质数加一，然后除以3，看看余数是否为0。经过计算，我们发现只有47和79满足这个条件。\n\n因此，大于40的第一个质数，使得这个质数加一能被3整除，是47。'}


In [48]:
router_chain.verbose = True

In [49]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是宇宙中极度紧密的区域，它的引力非常强大，以至于连光也无法逃脱。它是由大量物质坍缩而成，通常是恒星燃尽后坍缩形成的。黑洞的重力场非常强大，它会吸引周围的物质，甚至连光都无法逃脱。因此，我们无法直接观测到黑洞，只能通过它们对周围环境的影响来研究它们。黑洞被认为是宇宙中最神秘也是最奇特的天体之一，它们的研究对我们更深入地了解宇宙的起源和演化有着重要意义。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [51]:
print(chain.invoke("DNA复制的过程是怎样的，以及它在细胞分裂中扮演什么角色？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': 'DNA复制的过程是怎样的，以及它在细胞分裂中扮演什么角色？'}
> Finished chain.
{'input': 'DNA复制的过程是怎样的，以及它在细胞分裂中扮演什么角色？', 'text': '\n\nDNA复制是指细胞在分裂前将自身的DNA复制一份，以保证新生细胞能够拥有完整的遗传信息。这个过程发生在细胞的S期（DNA合成期），由多种酶和蛋白质协同作用完成。具体步骤如下：\n\n1. 起始点定位：在DNA双链的起始点，存在一段特殊的序列，称为起始点。该序列能够吸引一些酶和蛋白质结合，形成起始复合物。\n\n2. DNA双链解旋：起始复合物能够带动DNA双链解旋，使得双链分离，形成两条单链模板。\n\n3. RNA引物合成：在DNA双链分离的同时，一个酶叫做DNA依赖性RNA聚合酶（DNA'}


In [52]:
print(chain.invoke("什么是面向对象编程中的继承特性，它是如何在代码重用方面提供优势的？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '什么是面向对象编程中的继承特性，它是如何在代码重用方面提供优势的？'}
> Finished chain.
{'input': '什么是面向对象编程中的继承特性，它是如何在代码重用方面提供优势的？', 'text': '\n\n面向对象编程中的继承特性指的是一个类可以继承另一个类的属性和方法。这意味着子类可以使用父类的所有属性和方法，同时还可以定义自己独特的属性和方法。这种继承关系可以通过关键字"extends"来实现。\n\n继承在代码重用方面提供了很多优势。首先，它可以减少重复代码的编写，提高代码的可维护性。因为子类可以直接使用父类的方法，所以不需要重复编写相同的代码，只需要在子类中添加自己特有的代码即可。其次，继承也可以提高代码的可扩展性。如果需要在父类中添加新的方法或属性，那么所有的子类都会自动拥有这些新的特性，而不需要修改每个子类的代码。最后，继承也可以实现多'}


In [54]:
print(chain.invoke("宋词和唐诗在艺术特色和表现手法上有哪些主要差异？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '宋词和唐诗在艺术特色和表现手法上有哪些主要差异？'}
> Finished chain.
{'input': '宋词和唐诗在艺术特色和表现手法上有哪些主要差异？', 'text': '\n\n这个问题涉及到中国文学史上两个重要的文学时期，我需要更多时间来仔细研究并给出准确的回答。但是总的来说，宋词和唐诗都具有精练的语言、深刻的意境和独特的表现手法，但是在艺术特色和表现手法上也有一些不同之处。宋词注重写景抒情，多使用比兴和意象，追求意境的深远和细腻；而唐诗则更加注重抒发情感，多采用对偶和对仗，追求语言的雄浑和气势。此外，宋词还有明显的音乐性，而唐诗则更注重诗歌的节奏和韵律。总的来说，宋词和唐诗都'}
